### 함수 정리

#### 전처리 함수

In [1]:
def pretreatment(wav):  
    # 전처리 소수점 0.3 이하까지만
    y=[]
    for i in range(len(wav)):
        y.append(round(wav[i],3))
        
    # 0.02 이하는 0으로    
    t=[]
    for i in range(len(y)):
        if y[i]>=0.02 or  y[i]<=-0.02:
            t.append(y[i])
        else:
            y[i]=0
            t.append(y[i])
    
    # 파장 나누기
    re_time =[] # 파장 나눈 위치 저장 리스트
    count = 0 # 0의 개수 세기
    # and len(re_time)%2==0
    for i in range(len(t)):
        if t[i]==0:
            count+=1
        elif t[i]!=0 and count>=500:
            re_time.append(i)
            count=0
        else:
            count=0
    
    # 파장 길이 구하기
    l_list=[] # 파장 길이 저장 리스트 (x)
    l_count=0 # 0의 개수 카운트
    for i in range(len(re_time)):
        if i == len(re_time)-1:
            for j in range(len(t[re_time[i]:])):
                a = re_time[i]
                if t[a+j]!=0:
                    l_count+=1
            l_list.append(l_count)
            l_count=0
        else:
            for j in range(len(t[re_time[i]:re_time[i+1]])):
                a = re_time[i]
                if t[a+j]!=0:
                    l_count+=1
            l_list.append(l_count)
            l_count=0
            l_list.append

    # 두개로 나누기
    s_all=sum(l_list)/2
    b=0 # 각 파장 길이 중첩 변수
    c=0 # 몇번째 파장까지 나누는지 결정 변수
    for i in range(len(l_list)):
        b+=l_list[i]
        if len(l_list)==2:
            c=i+1
            break
    
        elif b >=s_all:
            c=i+1
            break
            
    global wav_01
    global wav_02
    # 각 나눈 파장 저장        
    wav_01 = wav[:re_time[c]]
    wav_02 = wav[re_time[c]:]



#### train

In [10]:
def train_file():
    import os
    import librosa
    import librosa.display
    import numpy as np
    import matplotlib.pyplot as plt
    import IPython.display
    import pandas as pd

    WAV_DIR = "./wav/wav/"
    
    x = []
    y = []
    # os.listdir(TRAIN_DIR) : train 폴더로부터 wav 파일을 하나씩 읽어온다

    # train
    for fname in os.listdir(WAV_DIR):
        try:
        # 읽은 파일이 wav 파일이 아닌 경우 다음으로 넘어감
            if ".wav" not in fname:
                pass
        
        # 라벨 데이터를 저장
        # text는 파일 명?에서 따올 것
            text =fname[:len(fname)-4]
            ts = text.split("_")
            y.append(ts[0])
            y.append(ts[1])
        
        # wav 파일 데이터를 읽어온다
            wav, sr = librosa.load(WAV_DIR+fname,sr=16000)
        
            pretreatment(wav)
        
        # mfcc 전환
            mfcc_01 = librosa.feature.mfcc(wav_01)
            mfcc_01 = librosa.feature.mfcc(wav_01, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
            mfcc_02 = librosa.feature.mfcc(wav_02)
            mfcc_02 = librosa.feature.mfcc(wav_02, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)      
        
            zero_pad_01 = np.zeros((100,400-len(mfcc_01[0])))
            mfcc_01 = np.hstack((mfcc_01,zero_pad_01)) 
        
            zero_pad_02 = np.zeros((100,400-len(mfcc_02[0])))
            mfcc_02 = np.hstack((mfcc_02,zero_pad_02)) 
        
        
        # wav 파일을 동일한 길이로 잘라서 특성 데이터로 저장
        # wav 데이터의 길이를 30000으로 설정
        #mfcc_01 = np.expand_dims(mfcc_01, -1)
        #mfcc_02 = np.expand_dims(mfcc_02, -1)
        
            x.append(mfcc_01)
            x.append(mfcc_02)
        except:
            print("파일 읽기 오류")
        # 다음으로 넘어간다
            raise
    global x_ex
    global y_en
    x = np.array(x)
    x_ex = np.expand_dims(x,-1)

    y = np.array(y)
    y_en = pd.get_dummies(y)

#### 모델 

In [15]:
def build_model(x_ex,y_en):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
    from tensorflow.keras.callbacks import EarlyStopping
    import os
    

    model = Sequential()

#CNN 층 : 이미지 데이터로부터 특징을 추출하는 전처리 (특성추출기)
#일반적으로 Conv2D는 2개나 3개와 MaxPooling2D 1개로 레이어를 구성
#컨볼루션 레이어는 데이터의 크기가 10x10 근처의 크기가 될때까지 반복
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(100,400,1), padding="same", activation="relu"))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),  padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),  padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # DNN에 넣어주기 위해서는 다차원 데이터를 1차원으로 펴주어야 한다
    model.add(Flatten())

    # 은닉층 1차원만 들어갈 수 있어서 Flatten()을 사용해 1차원으로 펴준다.
    model.add(Dense(256, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation="relu"))
    model.add(BatchNormalization())
    
    # 출력층
    model.add(Dense(len(y_en), activation="softmax"))
    model.add(BatchNormalization())
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    from tensorflow.keras.callbacks import ModelCheckpoint
#모델을 저장할 폴더
    MODEL_DIR = './model4/'

#폴더가 없다면 생성
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
    
#loss값에 따라서 파일명을 지을거임
    modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}-{accuracy:.4f}.hdf5'

#모델이 오차범위를 수정하면서 정확도가 높아지면 파일을 갱신할거임
    mc = ModelCheckpoint(filepath = modelpath, #filepath : 저장할 파일경로
                   monitor = 'accuracy', verbose = 1, #monitor : 어떤걸 모니터(체크)할건가 ,   val_loss: 테스트오차, val_acc : 테스트 정확도, loss : 훈련오차 // verboss : 진행내용 출력여부 1출력 0미출력
                   save_best_only = 'True') #save_best_only : 가장높은 수치반영 True : 이전정확도보다 높으면 저장 // False : 낮으면 저장

    h= model.fit(x_ex, y_en, validation_split = 0.3, epochs=500, callbacks=[mc])


#### 결과 텍스트 뽑는 중간 함수

#### 결과 함수

In [45]:
def model_result(fname):
    import os
    import librosa
    import librosa.display
    import numpy as np
    import matplotlib.pyplot as plt
    import IPython.display
    import pandas as pd
    
    TEST_REAL_DIR = "./wav/test_real/"
    X_real = []

    try:
        # 읽은 파일이 wav 파일이 아닌 경우 다음으로 넘어감
        if ".wav" not in fname:
            pass

            ####테스트 주석#####

            # 라벨 데이터를 저장
            # text는 파일 명?에서 따올 것
            #text =fname[:len(fname)-4]
            #ts = text.split("_")
            #print(ts)
            #y_test.append(ts[0])
            #y_test.append(ts[1])

            # wav 파일 데이터를 읽어온다
        wav, sr = librosa.load(TEST_REAL_DIR+fname,sr=16000)

        pretreatment(wav)

         # mfcc 전환
        mfcc_01 = librosa.feature.mfcc(wav_01)
        mfcc_01 = librosa.feature.mfcc(wav_01, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc_02 = librosa.feature.mfcc(wav_02)
        mfcc_02 = librosa.feature.mfcc(wav_02, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)      

        zero_pad_01 = np.zeros((100,400-len(mfcc_01[0])))
        mfcc_01 = np.hstack((mfcc_01,zero_pad_01)) 

        zero_pad_02 = np.zeros((100,400-len(mfcc_02[0])))
        mfcc_02 = np.hstack((mfcc_02,zero_pad_02)) 

            # wav 파일을 동일한 길이로 잘라서 특성 데이터로 저장
            # wav 데이터의 길이를 30000으로 설정
            #mfcc_01 = np.expand_dims(mfcc_01, -1)
            #mfcc_02 = np.expand_dims(mfcc_02, -1)

        X_real.append(mfcc_01)
        X_real.append(mfcc_02)

        X_real = np.array(X_real)

        X_real_ex = np.expand_dims(X_real, -1)

        from tensorflow.keras.models import load_model
        
        MODEL_DIR = "./model4/"
        model_list=[]
        # os.listdir(TRAIN_DIR) : train 폴더로부터 wav 파일을 하나씩 읽어온다

        # train
        file_dir =[]
        for fname in os.listdir(MODEL_DIR):
            try:
                if ".hdf5" not in fname:
                    pass
        
                text =fname[:]
                model_list.append(text)
                ts = text.split("-")
                file_dir.append(ts[2][:len(ts[2])-5])
            except:
                print("파일 읽기 오류")
                # 다음으로 넘어간다
                raise
        file_index = file_dir.index(max(file_dir))

        best=load_model('./model4/'+model_list[file_index])

        pre = best.predict(X_real_ex)
        
        #기존에 학습데이터와 후에 들어올 새로운 데이터의 예측값을 비교하기위 해 predict를 사용하여 변수화
        pre_ex = best.predict(x_ex)

        #새로 들어온 데이터 예측값
        b_ex = best.predict(X_real_ex)
        pre_num=[] # b_ex와 가장 가까운 pre_ex의 위치를 저장하기 위한 리스트
        num=10 # 0이랑 가까운 수를 찾기 위한 변수
        pre_number=0 # 위치 값을 pre_num에 저장하기 위한 변수
        for i in range(len(b_ex)):
            for j in range(len(pre_ex)):
                if num > np.sum((b_ex[i] - pre_ex[j])**2):
                    num = np.sum((b_ex[i] - pre_ex[j])**2)
                    pre_number=j
            pre_num.append(pre_number)

        y_en_np = np.array(y_en)
    
        global y_name
        y_name=[]
        for i in pre_num:
            for j in range(len(y_en)):
                if not y_en.iloc[i][j] == 0:
                    y_name.append(y_en.iloc[:,j].name[:2])
        y_name=' '.join(s for s in y_name)
    
        return y_name
        print(y_name)
    except:
        print("파일 읽기 오류")
            # 다음으로 넘어간다

In [17]:
train_file()
build_model(x_ex,y_en)

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 10.6451 - accuracy: 0.0000e+00
Epoch 00001: accuracy improved from -inf to 0.00000, saving model to ./model4\01-16.1181-0.0000.hdf5
1/1 [==============================] - 3s 3s/step - loss: 10.6451 - accuracy: 0.0000e+00 - val_loss: 16.1181 - val_accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 7.5161 - accuracy: 0.0625
Epoch 00002: accuracy improved from 0.00000 to 0.06250, saving model to ./model4\02-14.1033-0.0625.hdf5
1/1 [==============================] - 3s 3s/step - loss: 7.5161 - accuracy: 0.0625 - val_loss: 14.1033 - val_accuracy: 0.1250
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 7.4924 - accuracy: 0.0625
Epoch 00003: accuracy did not improve from 0.06250
1/1 [==============================] - 0s 175ms/step - loss: 7.4924 - accuracy: 0.0625 - val_loss: 16.1181 - val_accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - ETA: 0s - lo

In [46]:
model_result('고생3_했어3.wav')

'고생 했어'